# Домашнее задание: дообучение нейросети



Привет!

В этом домашнем задании вам предстоит самим дообучить сеть на новом датасете для классификации. 

https://stepik.org/lesson/392209/step/1?unit=381278

В последнем практическом занятии этого модуля мы решали задачу классификации кошек и собак с помощью предобученной на ImageNet сети. На самом деле, это не совсем честно, потому что среди классов ImageNet наверняка есть классы "кошка" и "собака", и сеть, обученная на ImageNet и без дообучения может классифицировать кошек и собак.
Поэтому в домашнем задании мы возьмем датасет фотографий Intel Image Classification (https://www.kaggle.com/puneet6060/intel-image-classification) природы.

![alt text](https://miro.medium.com/max/640/1*Y1y83HN7oI98EJT3LQgv-g.png)

## 1. Загрузка датасета


Для загрузки датасета из интернета воспользуемся командами bash. Bash -- язык командной строки linux. В ячейках jupyter notebook можно запускать команды bash, предварительно написав ! в начале ячейки.

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(os.path.join(dirname))

/kaggle/input
/kaggle/input/intel-image-classification
/kaggle/input/intel-image-classification/seg_train
/kaggle/input/intel-image-classification/seg_train/seg_train
/kaggle/input/intel-image-classification/seg_train/seg_train/mountain
/kaggle/input/intel-image-classification/seg_train/seg_train/street
/kaggle/input/intel-image-classification/seg_train/seg_train/buildings
/kaggle/input/intel-image-classification/seg_train/seg_train/sea
/kaggle/input/intel-image-classification/seg_train/seg_train/forest
/kaggle/input/intel-image-classification/seg_train/seg_train/glacier
/kaggle/input/intel-image-classification/seg_pred
/kaggle/input/intel-image-classification/seg_pred/seg_pred
/kaggle/input/intel-image-classification/seg_test
/kaggle/input/intel-image-classification/seg_test/seg_test
/kaggle/input/intel-image-classification/seg_test/seg_test/mountain
/kaggle/input/intel-image-classification/seg_test/seg_test/street
/kaggle/input/intel-image-classification/seg_test/seg_test/buildings
/

In [2]:
import warnings

warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)

In [3]:
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

# модули библиотеки PyTorch
import torch
from torchvision import datasets, transforms
# метрика качества
from sklearn.metrics import accuracy_score

# Проверяем, доступен ли GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


## Задание 1 (0 баллов)

Как обычно, начнем с загрузки даатсета в пайторч с помощью ImageFolder.

Для начала объявим трансформации.

Объявите трансформации для тренировочного и тестового датасета:

*   перевод картинки в тензор
*   нормализация с mean=[0.485, 0.456, 0.406] и std=[0.229, 0.224, 0.225]


Resize делать не надо! Все картинки этого датасета уже приведены к одному форматц 150*150

In [4]:
from torchvision import transforms

# Определяем трансформации для тренировочных и тестовых данных
transform_train = transforms.Compose([
    transforms.Resize((150, 150)),  # Изменяем размер изображений до 150x150
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

transform_val = transforms.Compose([
    transforms.Resize((150, 150)),  # Изменяем размер изображений до 150x150
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

И теперь объявим датасеты:

In [5]:
root_path = '/kaggle/input/intel-image-classification'
train_data = datasets.ImageFolder(root_path + "/seg_train/seg_train", transform=transform_train)
test_data = datasets.ImageFolder(root_path + "/seg_test/seg_test", transform=transform_val)

In [6]:
train_data

Dataset ImageFolder
    Number of datapoints: 14034
    Root location: /kaggle/input/intel-image-classification/seg_train/seg_train
    StandardTransform
Transform: Compose(
               Resize(size=(150, 150), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

Объявите даталоадеры. Помните, что для тренировочного даталоадера важно перемешивать данные, для тестового -- неважно. Батч сайз возьмите проивзольный. Но не берите слишком маленький, будете долго ждать обучения сети.

In [7]:
from torch.utils.data import random_split

# Размеры обучающей и валидационной выборок
train_size = int(0.8 * len(train_data))  # 80% данных для обучения
val_size = len(train_data) - train_size  # 20% данных для валидации

# Разделяем train_data на обучающую и валидационную выборки
train_subset, val_subset = random_split(train_data, [train_size, val_size])

# Создаем DataLoader для обучающей и валидационной выборок
batch_size = 256
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

Отлично! Теперь давайте посмотрим на примеры картинок из датасета, и заодно проверим, что вы объявили даталоадеры верно.

In [8]:
dataiter = iter(train_loader)
# батч картинок и батч ответов к картинкам
images, labels = next(dataiter)
images.shape, labels.shape

(torch.Size([256, 3, 150, 150]), torch.Size([256]))

In [9]:
def show_imgs(imgs, labels):
    f, axes= plt.subplots(1, 10, figsize=(30,5))
    for i, axis in enumerate(axes):
      axes[i].imshow(np.squeeze(np.transpose(imgs[i].numpy(), (1, 2, 0))), cmap='gray')
      axes[i].set_title(labels[i].numpy())
    plt.show()

show_imgs(images, labels)


Импортируем нужные модули для обучения сети:

In [10]:
import torch
# модуль, где определены слои для нейронных сетей
import torch.nn as nn
# модуль, где определены активайии для слоев нейронных сетей
import torch.nn.functional as F
from tqdm.auto import tqdm  # Автоматический выбор между консольным и notebook-режимом

## Задание 2. Обучение сети с нуля. (4 балла код + 1 балл вывод)

Построим сверточную нейронную сеть, обучим ее и посчитаем метрику accuracy на тестовой выборке.

Так как в этом датасете картинки разрешения 150*150 -- больше, чем разрешение картинок из датасета с практического занятия -- то давайте построим сеть с тремя сверточными слоями и двумя макспуллингами.

In [11]:
# Класс для преобразования 2D-тензора в 1D-вектор
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

# Класс сверточной нейронной сети
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # Первый сверточный слой: 3 входных канала (RGB), 5 выходных каналов, ядро 3x3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3, stride=1, padding=1)
        # Первый слой MaxPooling: ядро 2x2
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Второй сверточный слой: 5 входных каналов, 4 выходных канала, ядро 3x3
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=4, kernel_size=3, stride=1, padding=1)
        # Второй слой MaxPooling: ядро 2x2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Третий сверточный слой: 4 входных канала, 3 выходных канала, ядро 3x3
        self.conv3 = nn.Conv2d(in_channels=4, out_channels=3, kernel_size=3, stride=1, padding=1)
        
        # Преобразование карт активации в вектор
        self.flatten = Flatten()
        
        # Первый полносвязный слой: количество нейронов будет определено позже
        self.fc1 = nn.Linear(in_features=3 * 37 * 37, out_features=256)  # 37x37 после сверток и пулингов
        # Выходной полносвязный слой: 256 нейронов на входе, 6 нейронов на выходе (6 классов)
        self.fc2 = nn.Linear(in_features=256, out_features=6)
    
    def forward(self, x):
        # Первый сверточный слой + ReLU + MaxPooling
        x = self.pool1(F.relu(self.conv1(x)))
        # Второй сверточный слой + ReLU + MaxPooling
        x = self.pool2(F.relu(self.conv2(x)))
        # Третий сверточный слой + ReLU
        x = F.relu(self.conv3(x))
        
        # Преобразование карт активации в вектор
        x = self.flatten(x)
        
        # Первый полносвязный слой + ReLU
        x = F.relu(self.fc1(x))
        # Выходной полносвязный слой + Softmax
        x = F.softmax(self.fc2(x), dim=1)
        
        return x

Тут, как обычно, функция обучения сети (но модернизированная, чтобы выдавать лучшую модель и оставнавливать обучение, если скор падает):

In [12]:
def train(net, n_epoch=5, patience=3):
    # Выбираем функцию потерь
    loss_fn = torch.nn.CrossEntropyLoss()

    # Выбираем алгоритм оптимизации и learning_rate
    learning_rate = 1e-3
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

    # Переменные для ранней остановки
    best_accuracy = 0.0
    epochs_without_improvement = 0
    best_model_state = None

    # Обучаем сеть
    for epoch in tqdm(range(n_epoch), desc="Epochs"):
        net.train()  # Переводим модель в режим обучения
        running_loss = 0.0
        train_dataiter = iter(train_loader)
        
        for i, batch in enumerate(tqdm(train_dataiter, desc="Batches", leave=False)):
            # Получаем текущий батч
            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Обнуляем веса
            optimizer.zero_grad()

            # Forward pass (получение ответов на батч картинок)
            y_pred = net(X_batch)

            # Вычисление лосса от выданных сетью ответов и правильных ответов на батч
            loss = loss_fn(y_pred, y_batch)

            # Backpropagation (вычисление градиентов)
            loss.backward()

            # Обновление весов сети
            optimizer.step()

            # Выводим текущий loss
            running_loss += loss.item()

            # # Выводим качество каждые 10 батчей
            # if i % 10 == 9:
            #     print('[%d, %5d] loss: %.4f, acc: %.4f' %
            #           (epoch + 1, i + 1, running_loss / 10,
            #            accuracy_score(y_batch.cpu().numpy(), 
            #                           np.argmax(y_pred.detach().cpu().numpy(), axis=1))))
            #     running_loss = 0.0

        # Оценка модели на валидационной выборке
        net.eval()  # Переводим модель в режим оценки
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_batch in val_loader:
                X_val, y_val = val_batch
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_pred = net(X_val)
                _, predicted = torch.max(y_pred.data, 1)
                val_total += y_val.size(0)
                val_correct += (predicted == y_val).sum().item()

        val_accuracy = val_correct / val_total
        print(f'Epoch [{epoch + 1:02}/{n_epoch}], Validation Accuracy: {val_accuracy:.4f}')

        # Ранняя остановка
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            epochs_without_improvement = 0
            best_model_state = net.state_dict()  # Сохраняем веса лучшей модели
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f'Ранняя остановка: accuracy на валидации не улучшалась {patience} эпохи.')
            break

    # Загружаем веса лучшей модели
    net.load_state_dict(best_model_state)
    print('Обучение закончено')
    return net

In [13]:
max_epochs = 33

# Создаем модель и переносим ее на GPU
net = ConvNet().to(device)

# Обучаем сеть. Выберите количество эпох для обучения.
# Например, начните с 5 эпох и увеличьте, если сеть недообучена.
net = train(net, n_epoch=max_epochs)

Epochs:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [01/33], Validation Accuracy: 0.5162


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [02/33], Validation Accuracy: 0.5565


...

Epoch [14/33], Validation Accuracy: 0.6348


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [15/33], Validation Accuracy: 0.6423


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [16/33], Validation Accuracy: 0.6295
Ранняя остановка: accuracy на валидации не улучшалась 3 эпохи.
Обучение закончено


Посчитаем accuracy на test:

In [14]:
def evaluate_model(net, test_loader, device):
    """
    Оценивает точность модели на тестовой выборке.

    Параметры:
        net: Модель (нейронная сеть).
        test_loader: DataLoader для тестовой выборки.
        device: Устройство (GPU или CPU).

    Возвращает:
        accuracy: Точность модели на тестовой выборке.
    """
    net.eval()  # Переводим модель в режим оценки
    correct = 0
    total = 0

    with torch.no_grad():  # Отключаем вычисление градиентов
        for images, labels in tqdm(test_loader, desc="Batches", leave=False):
            # Переносим данные на GPU (или CPU)
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            
            # Считаем точность
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy on test set: {accuracy:.4f}')
    return accuracy

In [15]:
# Оцениваем модель на тестовой выборке
accuracy = evaluate_model(net, test_loader, device)
print(f'Точность модели: {accuracy * 100:.2f}%')

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Accuracy on test set: 0.6317
Точность модели: 63.17%


**Задание:**
Вывод:

Проанализирейте: переобучилась ли ваша сеть? Достаточно ли хорошо она предсказывает картинки на тесте?

**После доработки тренировочной функции модель не может переобучиться, но без неё, любая подель переобучится и можно считать, что эта переобучилась. Предсказания самодельной модели не очень, всего 63%**

## Задание 3. Дообучение AlexNet (2+2 балла код + 1 балл вывод)

Теперь давайте попробуем дообучить сеть, предобученную на ImageNet, на нашем датасете.

На практическом занятии мы дообучали Vgg16, теперь даватйе возьмем другую сеть -- AlexNet.

Список сетей, предобученных на ImageNet, которые есть в библиотеке PyTorch: https://pytorch.org/docs/stable/torchvision/models.html

In [16]:
from torchvision import models
AlexNet = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 226MB/s] 


In [17]:
AlexNet.parameters

<bound method Module.parameters of AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=92

**Задание 3.1:**

Напишите класс New_AlexNet на основе AlexNet, у которой заменили последний слой. Заморозьте все слои, кроме слоев классификатора (всех линейных слоев)

In [18]:
class New_AlexNet(nn.Module):
    def __init__(self, num_classes=6):  # 6 классов в нашем датасете
        super(New_AlexNet, self).__init__()
        
        # Загружаем предобученную AlexNet
        self.alexnet = models.alexnet(pretrained=True)
        
        # Замораживаем все слои, кроме слоев классификатора
        for param in self.alexnet.parameters():
            param.requires_grad = False
        
        # Заменяем последний слой классификатора
        # В AlexNet классификатор находится в self.alexnet.classifier
        # Последний слой — это шестой элемент (индекс 6) в self.alexnet.classifier
        # Изначально он имеет 1000 выходов (по количеству классов в ImageNet)
        # Мы заменяем его на слой с num_classes выходами
        self.alexnet.classifier[6] = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        # Forward pass через AlexNet
        x = self.alexnet(x)
        return x

In [19]:
netA = New_AlexNet().to(device)
netA = train(netA, n_epoch=max_epochs)

Epochs:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [01/33], Validation Accuracy: 0.8867


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [02/33], Validation Accuracy: 0.8910


...

Epoch [05/33], Validation Accuracy: 0.8931


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [06/33], Validation Accuracy: 0.8956


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [07/33], Validation Accuracy: 0.8967
Ранняя остановка: accuracy на валидации не улучшалась 3 эпохи.
Обучение закончено


И посмотрим на скор на тесте:

In [20]:
# Оцениваем модель на тестовой выборке
accuracy = evaluate_model(netA, test_loader, device)
print(f'Точность модели: {accuracy * 100:.2f}%')

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Accuracy on test set: 0.8917
Точность модели: 89.17%


**Задание 3.2:**

Напишите класс New_AlexNet на основе AlexNet, у которой заменили последний слой(как в задании выше) Заморозьте все слои, кроме **двух последних** слоев классификатора

In [21]:
class New_AlexNet_v2(nn.Module):
    def __init__(self, num_classes=6):  # 6 классов в вашем датасете
        super(New_AlexNet_v2, self).__init__()
        
        # Загружаем предобученную AlexNet
        self.alexnet = models.alexnet(pretrained=True)
        
        # Замораживаем все слои, кроме двух последних слоев классификатора
        for param in self.alexnet.parameters():
            param.requires_grad = False
        
        # Размораживаем два последних слоя классификатора
        for param in self.alexnet.classifier[4].parameters():  # Пятый слой (индекс 4)
            param.requires_grad = True
        for param in self.alexnet.classifier[6].parameters():  # Седьмой слой (индекс 6)
            param.requires_grad = True
        
        # Заменяем последний слой классификатора
        # В AlexNet классификатор находится в self.alexnet.classifier
        # Последний слой — это шестой элемент (индекс 6) в self.alexnet.classifier
        # Изначально он имеет 1000 выходов (по количеству классов в ImageNet)
        # Мы заменяем его на слой с num_classes выходами
        self.alexnet.classifier[6] = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        # Forward pass через AlexNet
        x = self.alexnet(x)
        return x

In [22]:
netA2 = New_AlexNet_v2().to(device)
netA2 = train(netA2, n_epoch=max_epochs)

Epochs:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [01/33], Validation Accuracy: 0.8917


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [02/33], Validation Accuracy: 0.9027


...

Epoch [03/33], Validation Accuracy: 0.9006


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [04/33], Validation Accuracy: 0.8960


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch [05/33], Validation Accuracy: 0.8938
Ранняя остановка: accuracy на валидации не улучшалась 3 эпохи.
Обучение закончено


И посмотрим на скор на тесте:

In [23]:
# Оцениваем модель на тестовой выборке
accuracy = evaluate_model(netA2, test_loader, device)
print(f'Точность модели: {accuracy * 100:.2f}%')

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Accuracy on test set: 0.8950
Точность модели: 89.50%


**Задание 3.3:** Вывод:

Какие результаты у вас получились? Классифицирует ли картинки природы лучше сеть, предобученная на ImageNet, или ваша сеть из задания два получилась лучше?
Есть ли разница между дообучением двух последних слоев сети или всех линейных слоев сети?

 **Преобученная сеть AlexNet показала результат лучше, чем самодельная сетка, причем, у второго варианта AlexNet точность чуть получше на доли процента. Здесь большую проль сыграла доработка функции обучения, что она вовремя останавливает обучение и возвращает модель с наилучшим скором не валидационном наборе.**